# People in Public 175K Dataset

In this example, we'll load the [People in Public 175K Dataset](https://visym.github.io/collector/pip_175k) from [Visym Labs](https://www.visym.com/site_0820/index.html) into FiftyOne.

Per the dataset homepage, PIP-175K contains 184,379 video clips of 68 classes of activities performed by people in public places. The activity labels are subsets of the 37 activities in the [Multiview Extended Video with Activities (MEVA) dataset](https://mevadata.org) and is consistent with the [Activities in Extended Video (ActEV) challenge](https://actev.nist.gov).

In [3]:
%%html
<iframe width="800" height="450" src="https://www.youtube.com/embed/HjNa7_T-Xkc?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>

## Download dataset

The dataset can be downloaded from [this page](https://visym.github.io/collector/pip_175k) via [this link (55.3GB)](https://dl.dropboxusercontent.com/s/xwiacwo9y5uci9v/pip_175k.tar.gz).

In [ ]:
# Download and unpack the dataset
!wget https://dl.dropboxusercontent.com/s/xwiacwo9y5uci9v/pip_175k.tar.gz
!tar -xvzf pip_175k.tar.gz
!rm pip_175k.tar.gz

After downloading, you'll have a `pip_175k/` directory with the following contents:

```
pip_175k/
    videos/
        car_starts/
            <uuid1>.mp4
            ...
        person_transfers_object_to_person/
            <uuid2>.mp4
            ...
        ...
    trainset.pkl
    testset.pkl
    valset.pkl
    ...
```

The `videos/` subdirectory contains the videos files organized as a directory tree that encodes the primary activity in each video.

The `.pkl` files contain dense 2D bounding annotations + additional activity labels for each video, stored in [VIPY format](https://github.com/visym/vipy).

We'll need to install the YIPY package in order to load the dense annotations:

In [ ]:
# Install VIPY package
!pip install vipy

## Installing FiftyOne

You can install FiftyOne and the necessary dependencies, if necessary, as follows:

In [ ]:
# Install FiftyOne
!pip install --index https://pypi.voxel51.com fiftyone

# We'll need ffmpeg to work with video datasets
!sudo apt-get install -y ffmpeg

## Quick preview

FiftyOne provides native support for visualizing datasets stored as [video classification directory trees](https://voxel51.com/docs/fiftyone/user_guide/dataset_creation/datasets.html#videoclassificationdirectorytree) on disk, like the `pip_175k/videos/` sudirectory of the PIP-175K dataset.

Therefore, you can preview a random subset of the dataset as follows:

In [ ]:
# Path to your copy of PIP-175K
PIP_175K_DIR = "/path/to/pip_175k"

In [ ]:
import os
import fiftyone as fo

# Load 100 random videos
dataset = fo.Dataset.from_dir(
    os.path.join(PIP_175K_DIR, "videos"),
    fo.types.VideoClassificationDirectoryTree,
    name="PIP-175K-sample",
    shuffle=True,
    max_samples=100,
)

# Visualize in the FiftyOne App
session = fo.launch_app(dataset)

![pip-175k-sample-gif](https://user-images.githubusercontent.com/25985824/97036100-50eb9b00-1535-11eb-8e9b-a939aba87b5b.gif)

<!--- ![pip-175k-sample](https://user-images.githubusercontent.com/25985824/97035129-e423d100-1533-11eb-809a-8c7da01e7d4c.png) --->

## Loading the full annotations

We can load the complete annotations from the VIPY `.pkl` files by [writing a custom DatasetImporter](https://voxel51.com/docs/fiftyone/user_guide/dataset_creation/datasets.html#writing-a-custom-datasetimporter):

In [ ]:
from collections import defaultdict
import logging
import os

import vipy

import eta.core.utils as etau
import eta.core.video as etav

import fiftyone as fo
import fiftyone.utils.data as foud


logger = logging.getLogger(__name__)


class VIPYDatasetImporter(foud.LabeledVideoDatasetImporter):
    """Importer for labeled video datasets stored in
    `VIPY format <https://github.com/visym/vipy>`_.

    Args:
        dataset_dir: the dataset directory
        pkl_file (None): the name of the ``.pkl`` file within ``dataset_dir``
            from which to load samples + annotations
        shuffle (False): whether to randomly shuffle the order in which the
            samples are imported
        seed (None): a random seed to use when shuffling
        max_samples (None): a maximum number of samples to import. By default,
            all samples are imported
    """

    def __init__(
        self,
        dataset_dir,
        pkl_file=None,
        shuffle=False,
        seed=None,
        max_samples=None,
    ):
        if pkl_file is None:
            pkl_paths = etau.get_glob_matches(
                os.path.join(dataset_dir, "*.pkl")
            )
            pkl_file = os.path.basename(pkl_paths[0])

        super().__init__(
            dataset_dir, shuffle=shuffle, seed=seed, max_samples=max_samples
        )

        self.pkl_file = pkl_file
        self._pkl_path = os.path.join(dataset_dir, pkl_file)
        self._samples = None
        self._iter_samples = None
        self._num_samples = None

    def __iter__(self):
        self._iter_samples = iter(self._samples)
        return self

    def __len__(self):
        return self._num_samples

    def __next__(self):
        v = next(self._iter_samples)
        return _parse_vipy_video(v)

    @property
    def has_dataset_info(self):
        return False

    @property
    def has_video_metadata(self):
        return False

    @property
    def label_cls(self):
        return fo.Classifications

    @property
    def frame_labels_cls(self):
        return fo.Detections

    def setup(self):
        logger.info("Loading VIPY pkl '%s'...", self._pkl_path)
        pip = vipy.util.load(self._pkl_path)
        logger.info("Loading complete")

        self._samples = self._preprocess_list(pip)
        self._num_samples = len(self._samples)


def _parse_vipy_video(v):
    video_path = v.filename()

    video_metadata = fo.VideoMetadata.build_for(video_path)
    width = video_metadata.frame_width
    height = video_metadata.frame_height

    # Activities
    activities = fo.Classifications(
        classifications=[
            fo.Classification(label=a.category())
            for a in v.activities().values()
        ]
    )

    # Detections
    frames = defaultdict(lambda: defaultdict(fo.Detections))
    for track in v.tracks().values():
        label = track.category()
        for frame_number in range(track.startframe(), track.endframe() + 1):
            x, y, w, h = track[frame_number].to_xywh()
            bounding_box = [x / width, y / height, w / width, h / height]
            detection = fo.Detection(label=label, bounding_box=bounding_box)
            frames[frame_number + 1]["objects"].detections.append(detection)

    return video_path, None, activities, frames

We can then use the `VIPYDatasetImporter` to load samples with their full annotations into FiftyOne:

In [ ]:
# Make an importer that will load 100 random samples
importer = VIPYDatasetImporter(
    PIP_175K_DIR,
    pkl_file="valset.pkl",
    shuffle=True,
    max_samples=100,
)

# Load samples into FiftyOne dataset
dataset = fo.Dataset.from_importer(
    importer,
    label_field="gt",
    name="PIP-175K-sample-with-detections",
)

# Visualize samples in the App
session = fo.launch_app(dataset)

![pip-175k-sample-with-detections-gif](https://user-images.githubusercontent.com/25985824/97036117-5517b880-1535-11eb-90fd-10fcf95446a6.gif)

<!--- ![pip-175k-sample-with-detections](https://user-images.githubusercontent.com/25985824/97035134-e4bc6780-1533-11eb-93ef-e33a184dc015.png) --->

![pip-175k-sample-with-detections-expanded](https://user-images.githubusercontent.com/25985824/97035121-e0904a00-1533-11eb-9e4b-f6c961a8ff1e.png)

## Exploring the dataset

With the data in FiftyOne, we can now explore the dataset using [dataset views](https://voxel51.com/docs/fiftyone/user_guide/using_views.html).

For example, we can filter the dataset to only show videos with label `person_exits_car`:

In [ ]:
from fiftyone import ViewField as F

# Create a view that contains only videos with label `person_exits_car`
view = dataset.filter_classifications(
    "gt", F("label") == "person_exits_car", only_matches=True
)

# Show view in App
session.view = view

![person_exits_car](https://user-images.githubusercontent.com/25985824/97035137-e554fe00-1533-11eb-8805-6a8abcdf54fe.png)